# Soils Revealed precalculations with `Zarrs`

<a id='libraries'></a>
### Python libraries

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from affine import Affine
import xarray as xr
import dask.array as da
from dask.diagnostics import ProgressBar, Profiler, ResourceProfiler, CacheProfiler, visualize
from xhistogram.xarray import histogram
from rasterio import features
import zarr
import rioxarray
import regionmask
import gcsfs
from geocube.api.core import make_geocube
import shapely.wkb 
from shapely.ops import cascaded_union
import json
import requests
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from tqdm import tqdm

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from pathlib import Path 
env_path = Path('.') / '.env'

In [ ]:
from dask.distributed import Client
client = Client()  # start distributed scheduler locally.  Launch dashboard
client

<a id='utils'></a>
### Utils

<a id='df_from_carto'></a>
**df_from_carto**

In [ ]:
def df_from_carto(account, query):
    """
    It gets data by querying a carto table and converts it into a GeoDataFrame.
    """
    urlCarto = f"https://{account}.carto.com/api/v2/sql"
    
    sql = {"q": query}
    r = requests.get(urlCarto, params=sql)
    
    data = r.json()
    
    df = gpd.GeoDataFrame(data.get("rows"))
    if 'the_geom' in df.columns:
        # Change geometry from WKB to WKT format
        df['geometry'] = df.apply(lambda x: shapely.wkb.loads(x['the_geom'],hex=True), axis=1 )
        df.drop(columns=['the_geom'], inplace=True)
        if 'the_geom_webmercator' in df.columns:
            df.drop(columns=['the_geom_webmercator'], inplace=True)
        df.crs = {'init': 'epsg:4326'}
        df = df.to_crs({'init': 'epsg:4326'})
        
    return df

**intersect_areas**

In [ ]:
def intersect_areas(gdf, geometry):
    """
    Intersection between the areas of a GeoDataFrame and a geometry
    """
    sindex = gdf.sindex
    
    # Areas that intersect with the geometry
    possible_matches_index = list(sindex.intersection(geometry.bounds))
    possible_matches = gdf.iloc[possible_matches_index]
    
    # Intersection between the areas and the geometry
    precise_matches = possible_matches.intersection(geometry)
    
    # Replace areas with the intersected ones
    final_matches = possible_matches[~precise_matches.is_empty]
    final_matches['geometry'] = list(precise_matches[~precise_matches.is_empty])
    
    return final_matches

**plot_hist**

In [ ]:
def plot_hist(x_min, count):
    width = x_min[1]-x_min[0]
    width -= width/5.
    x_min += width/(5.*2)
    per = count/count.sum()*100
    
    plt.figure(figsize=(10,5))
    
    plt.bar(x_min, per, width=width)
    
    plt.plot([0,0], [0,per.max()], color = 'k', linestyle = '--')
    
    plt.title('Soil Organic Carbon Stock')
    plt.xlabel('SOC stock t C/ha)')
    plt.ylabel('(%) of total area')

**set_lat_lon_attrs**

In [ ]:
def set_lat_lon_attrs(ds):
    """ Set CF latitude and longitude attributes"""
    ds["lon"] = ds.lon.assign_attrs({
      'axis' : 'X',
       'long_name' : 'longitude',
        'standard_name' : 'longitude',
         'stored_direction' : 'increasing',
          'type' : 'double',
           'units' : 'degrees_east',
            'valid_max' : 360.0,
             'valid_min' : -180.0
             })
    ds["lat"] = ds.lat.assign_attrs({
      'axis' : 'Y',
       'long_name' : 'latitude',
        'standard_name' : 'latitude',
         'stored_direction' : 'increasing',
          'type' : 'double',
           'units' : 'degrees_north',
            'valid_max' : 90.0,
             'valid_min' : -90.0
             })
    return ds

**create_ds_mask**

In [ ]:
def create_ds_mask(df, ds, id_prop, name_prop, name, debug=False):
  
    # Get mean ds cell area (in degrees) 
    mean_y_size = np.diff(ds.lat.values).mean()
    #print(mean_y_size)
    mean_x_size = np.diff(ds.lat.values).mean()
    #print(mean_x_size)
    mean_area = mean_y_size * mean_x_size
    print(f"The mean ds cell area is {np.round(mean_area, 6)} deg.\n")
    
    # Clip gdf to bounding box of ds
    xmin = ds.lon.min().values.tolist()
    xmax = ds.lon.max().values.tolist()
    ymin = ds.lat.min().values.tolist()
    ymax = ds.lat.max().values.tolist()
    df = df.cx[xmin:xmax, ymin:ymax]
    
    
    # Add area of geoms to gdf
    df = df.assign(area = df.area)
    df = df.assign(area_is_gt_cell = df['area'] > mean_area)
    print(f"Clipped gdf to dataset bounds, giving {len(df[id_prop])} potential geometries, of which {df['area_is_gt_cell'].sum()} are large enough.\n")
    
    print("Geometries smaller than mean cell size:")
    print(df.loc[df['area_is_gt_cell'] == False, [id_prop, name_prop]])
    print("\n")

    # Extract geoms, codes, names, and create code_ints that are large enough!
    id_names = df.loc[df['area_is_gt_cell'] == True, name_prop].values
    if debug:
        print("\n NAMEs for mask:\n")
        print(len(id_names))
        print(id_names)
    id_codes = df.loc[df['area_is_gt_cell'] == True, id_prop].values
    if debug:
        print("\n CODEs for mask:\n")
        print(len(id_codes))
        print(id_codes)
    geoms = df.loc[df['area_is_gt_cell'] == True, 'geometry'].values
    id_ints = list(range(0, len(id_codes)))
    if debug:
        print("\n IDs for mask:\n")
        print(len(id_ints))
        print(id_ints)

    # create mask object
    da_mask = regionmask.Regions(
      name = name,
      numbers = id_ints,
      names = id_names,
      abbrevs = id_codes,
      outlines = geoms)\
      .mask(ds)\
      .rename(name)
    if debug:
        print(da_mask)

    # get the ints actually written to mask
    id_ints_mask = da_mask.to_dataframe().dropna()[name].unique()
    id_ints_mask = np.sort(id_ints_mask).astype('int')
    if debug:
        print("\n Actually IDs on mask:\n")
        print(len(id_ints_mask))
        print(id_ints_mask)
    
    # resample the ints, names and codes
    id_names_mask = id_names[id_ints_mask]
    if debug:
        print("\n Actually NAMES on mask:\n")
        print(len(id_names_mask))
        print(id_names_mask)
    id_codes_mask = id_codes[id_ints_mask]
    if debug:
        print("\n Actually CODES on mask:\n")
        print(len(id_codes_mask))
        print(id_codes_mask)
    print(f"Finished writing {len(id_codes_mask)} geometries to {name} mask.\n")
    
    # update da attributes
    da_mask.attrs['id_ints'] = id_ints_mask
    da_mask.attrs['id_codes'] = id_codes_mask  
    da_mask.attrs['id_names'] = id_names_mask
    da_mask = set_lat_lon_attrs(da_mask)
    return da_mask

**precalculate_change**

In [ ]:
def precalculate_change(df, xds, nBinds=40, bindsRange=[-50, 50]):
    indexes = xds.index.attrs.get('id_ints').astype(np.float32)
    times = xds.coords.get('time').values
    depths = xds.coords.get('depth').values
    
    Indexes = []
    Counts = []
    Bins = []
    Change = []
    Depth = []
    Dates = []
    
    for index in tqdm(indexes):
        xds_index = xds.where(xds['index'].isin(index))
        for depth in depths:
            for i in range(len(times)):
                for j in range(len(times)-1-i):
                    start_date = times[i]
                    end_date = times[i+j+1]
                    
                    # Get difference between two dates
                    diff = xds_index.loc[dict(time=end_date, depth=depth)] - xds_index.loc[dict(time=start_date, depth=depth)]
                    
                    # Get counts and binds of the histogram
                    h, bins = da.histogram(diff.stocks, bins=nBinds, range=bindsRange)
                    
                    # Compute change value
                    start_year = pd.to_datetime(start_date).year
                    end_year = pd.to_datetime(end_date).year
                    dYears = end_year - start_year
                    mean_diff = diff.stocks.mean(skipna=True).compute().values
                    change = mean_diff/dYears
                    
                    # Save values
                    Indexes.append(int(index))
                    Counts.append(h.compute())
                    Bins.append(bins)
                    Change.append(change)
                    Depth.append(depth)
                    Dates.append([start_year, end_year])
                    
    
    df_change = pd.DataFrame({"index": Indexes, "counts": Counts, "bins": Bins, "change":Change, "depth": Depth, "years": Dates})
    
    return pd.merge(df.drop(columns='geometry').reset_index(drop=True).reset_index(), 
                    df_change, 
                    how='left', 
                    on='index').drop(columns=['index'])

## Read `xarray.Dataset` from `Zarr` in Google cloud storage

In [ ]:
%%time
# Connect to GS
project_name = 'soc-platform'
bucket_name = 'vizz-data-transfer'
root = bucket_name+'/SOC_maps/soil-data.zarr'
group = 'experimental-dataset-stock'
private_key = json.loads(os.getenv("PRIVATE_KEY"))

gc = gcsfs.GCSFileSystem(project=project_name, token=private_key)
store = gc.get_mapper(root)

# Return zarr group
xds = xr.open_zarr(store=store, group=group, consolidated=True)

# Change coordinates names
xds = xds.rename({'x': 'lon', 'y': 'lat'})

# Change depth coord from 0 to 1 dimensional array
depths = xds.coords.get('depth').values
if depths.ndim == 0: 
    xds = xds.squeeze().drop("depth")
    xds = xds.assign_coords({"depth": np.array([depths])})

xds

## Read vector data
**Political boundaries**

In [ ]:
def prepare_vector_data(iso=None, tolerance=None):
    # Read Political boundaries:
    print('Reading Political boundaries')
    gdf_pb = gpd.read_file('../data/mbtiles/gadm36_political_boundaries/gadm36_political_boundaries.shp')
    # Select up to level 1 admin areas
    gdf_pb = gdf_pb[gdf_pb['level'] <= 1]
    
    # Read Landforms
    print('Reading Landforms')
    gdf_land = gpd.read_file('../data/mbtiles/ne_10m_geography_regions_polys/ne_10m_geography_regions_polys.shp')
    
    # Read Biomes
    print('Reading Biomes')
    gdf_bio = gpd.read_file('../data/mbtiles/bio_042_ecoregions_by_biome_1_14/bio_042_ecoregions_by_biome_1_14.shp')
    
    # Read Hydrological basins
    print('Reading Hydrological basins')
    gdf_hb = gpd.read_file('../data/mbtiles/hydrological_basins/hydrological_basins.shp')
    #Make valid hydrological basin geometries
    gdf_hb['geometry'] = gdf_hb['geometry'].apply(lambda x: x.buffer(0))

    vector_data = {'political_boundaries': gdf_pb, 'landforms': gdf_land, 'biomes': gdf_bio, 'hydrological_basins': gdf_hb}
    
    if iso:
        print('Intersecting areas with the selected country')
        gdf_pb = gdf_pb[gdf_pb['gid_0'] == iso]
        if tolerance:
            gdf_pb['geometry'] = gdf_pb['geometry'].apply(lambda x: x.simplify(tolerance)) 
            
        vector_data['political_boundaries'] = gdf_pb
        
        country = gdf_pb[gdf_pb['level'] == 0]['geometry'].iloc[0].buffer(0)
        
        for data_name in list(vector_data.keys())[1:]:
            print(data_name)
            vector_data[data_name] = intersect_areas(vector_data[data_name], country)
            
    # Set index
    for data_name in list(vector_data.keys()):
        vector_data[data_name] = vector_data[data_name].reset_index(drop=True).reset_index()
        
    return vector_data

In [ ]:
vector_data = prepare_vector_data(iso='ARG', tolerance=0.075)

In [ ]:
account = "wri-01"
query = "SELECT name_0, name_1, gid_0, gid_1, the_geom FROM gadm36_political_boundaries WHERE level = 1 and gid_0 = 'ARG'"

df = df_from_carto(account, query)

df = df.iloc[10:12]
#df['geometry'] = df['geometry'].apply(lambda x: x.simplify(0.05))

fig, ax = plt.subplots(figsize=(16,8.5))
df.plot(ax=ax)

## Zonal statistics

**Create the data mask by rasterizing the vector data**

In [ ]:
def create_ds_mask_2(df, ds, name):
    
    # Create index column
    if 'index' not in df:
        df.reset_index(drop=True).reset_index()
    
    # Get mean ds cell area (in degrees) 
    mean_y_size = np.diff(ds.lat.values).mean()
    #print(mean_y_size)
    mean_x_size = np.diff(ds.lat.values).mean()
    #print(mean_x_size)
    mean_area = mean_y_size * mean_x_size
    print(f"The mean ds cell area is {np.round(mean_area, 6)} deg.\n")
    
    # Clip gdf to bounding box of ds
    xmin = ds.lon.min().values.tolist()
    xmax = ds.lon.max().values.tolist()
    ymin = ds.lat.min().values.tolist()
    ymax = ds.lat.max().values.tolist()
    df = df.cx[xmin:xmax, ymin:ymax]
    
    
    # Add area of geoms to gdf
    df = df.assign(area = df.area)
    df = df.assign(area_is_gt_cell = df['area'] > mean_area)
    print(f"Clipped gdf to dataset bounds, giving {len(df['index'])} potential geometries, of which {df['area_is_gt_cell'].sum()} are large enough.\n")
    
    print("Geometries smaller than mean cell size:")
    print(df.loc[df['area_is_gt_cell'] == False, ['index']])
    print("\n")

    # Extract indexes and geoms that are large enough!
    id_ints = df.loc[df['area_is_gt_cell'] == True, 'index'].values
    geoms = df.loc[df['area_is_gt_cell'] == True, 'geometry'].values
    
    print(f'Number of indexes: {len(id_ints)}')
    print(f'Number of geoms: {len(geoms)}')

    # create mask object
    da_mask = regionmask.Regions(
      name = name,
      numbers = id_ints,
      outlines = geoms)\
      .mask(ds)\
      .rename(name)

    # get the ints actually written to mask
    id_ints_mask = da_mask.to_dataframe().dropna()[name].unique()
    id_ints_mask = np.sort(id_ints_mask).astype('int')
    
    print(f'Number of ints ints mask: {len(id_ints_mask)}')
    
    # update da attributes
    da_mask.attrs['id_ints'] = id_ints_mask
    da_mask = set_lat_lon_attrs(da_mask)
    return da_mask, id_ints, geoms, df

In [ ]:
name='landforms'
da_mask, id_ints, geoms, df = create_ds_mask_2(vector_data[name], xds, name)

In [ ]:
id_ints

In [ ]:
df

In [ ]:
da_mask.to_dataframe().dropna()[name].unique()

In [ ]:
geoms[5].area

In [ ]:
da_mask = regionmask.Regions(
      name = name,
      numbers = id_ints[:4],
      outlines = geoms[:4])\
      .mask(xds)\
      .rename(name)

In [ ]:
regions = regionmask.Regions(
    name = name, 
    numbers = id_ints[:3], 
    outlines = geoms[:3].buffer(5))
da_mask = regions.mask(xds, wrap_lon=False).rename(name)

In [ ]:
da_mask.to_dataframe().dropna()[name].unique()

In [ ]:
xds

In [ ]:
test = xds.copy()
test["index"] = da_mask
test = test.where(test.index == 0.)

In [ ]:
import cartopy.crs as ccrs
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.add_geometries(regions.polygons, ccrs.PlateCarree(), fc="none", ec="0.1")
test.isel(time=1).stocks.plot.pcolormesh(ax=ax)

In [ ]:
da_masks = {}
for name in vector_data.keys():
    print(f'Creating xarray DataArray mask for {name}:')
    da_masks[name] = create_ds_mask_2(vector_data[name], xds, name) 

In [ ]:
da_masks['landforms'].to_dataframe().dropna()['landforms'].unique()

In [ ]:
vector_data['landforms']['geometry'].iloc[1]

In [ ]:
%%time
name = 'Argentina'
name_prop = 'name_1'
id_prop = 'gid_1'

da_mask = create_ds_mask(df, xds, id_prop, name_prop, name, debug=False)
da_mask

In [ ]:
fig, ax = plt.subplots(figsize=(16,8.5))
ax.imshow(da_mask.values)

**Create a `xarray.Dataset` by merging both `xarray.Datasets`**

In [ ]:
xds["index"] = da_mask
xds

### Histogram
#### **Example**
Select subsample:

In [ ]:
xds = xds.isel(time=[0,18,35])

In [ ]:
indexes = xds.index.attrs.get('id_ints').astype(np.float32)
times = xds.coords.get('time').values
depths = xds.coords.get('depth').values

In [ ]:
xds_index = xds.where(xds['index'].isin(indexes[0]))
diff = xds_index.loc[dict(time=times[2], depth=depths[0])] - xds_index.loc[dict(time=times[0], depth=depths[0])]
diff

**with `xhistogram`**

In [ ]:
bins = np.linspace(-50, 50, 41)
h = histogram(diff.stocks, bins=[bins], dim=['lat', 'lon'])
h

In [ ]:
with ProgressBar():
    h.plot()

Plot change distribution

In [ ]:
with ProgressBar():
    count = h.values
    
x_min = bins[:-1]
plot_hist(x_min, count)

**with `da.histogram`**

In [ ]:
h, bins = da.histogram(diff.stocks, bins=40, range=[-50, 50])

Plot change distribution

In [ ]:
with ProgressBar():
    count = h.compute()

x_min = bins[:-1]
plot_hist(x_min, count)

**Display change value**

In [ ]:
years = pd.to_datetime(times[2]).year - pd.to_datetime(times[0]).year
mean_diff = diff.stocks.mean(skipna=True).compute().values
change = mean_diff/years

In [ ]:
print(f'Soil Organic Carbon Stock Change: {change} t C/ha year')

#### **Entire` DataFrame`**

In [ ]:
%%time
with ProgressBar():
    result = precalculate_change(df, xds, nBinds=40, bindsRange=[-50, 50])

In [ ]:
result

**Plot change distribution**

In [ ]:
index = 0
count = result['counts'].iloc[index].copy()
bins = result['bins'].iloc[index].copy()
change = result['change'].iloc[index].copy()
x_min = bins[:-1]
per = count/count.sum()*100
print(f'Soil Organic Carbon Stock Change: {change} t C/ha year')
plot_hist(x_min, count)

### Mean value
**Example**

In [ ]:
grouped_xds = xds.groupby(xds.index)
grid_mean = grouped_xds.mean().rename({"stocks": "mean"})
grid_mean.to_dataframe()

In [ ]:
def transform_from_latlon(lat, lon):
    lat = np.asarray(lat)
    lon = np.asarray(lon)
    trans = Affine.translation(lon[0], lat[0])
    scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
    return trans * scale

def rasterize(shapes, coords, latitude='latitude', longitude='longitude',
              fill=np.nan, **kwargs):
    """Rasterize a list of (geometry, fill_value) tuples onto the given
    xray coordinates. This only works for 1d latitude and longitude
    arrays.
    """
    transform = transform_from_latlon(coords[latitude], coords[longitude])
    out_shape = (len(coords[latitude]), len(coords[longitude]))
    raster = features.rasterize(shapes, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float, **kwargs)
    spatial_coords = {latitude: coords[latitude], longitude: coords[longitude]}
    return xr.DataArray(raster, coords=spatial_coords, dims=(latitude, longitude))

In [ ]:
gdf_pb = gpd.read_file('../data/mbtiles/gadm36_political_boundaries/gadm36_political_boundaries.shp')
# Select up to level 1 admin areas
gdf_pb = gdf_pb[gdf_pb['level'] == 1]
gdf_pb = gdf_pb[gdf_pb['gid_0'] == 'ARG']

In [ ]:
gdf_pb = gdf_pb.reset_index(drop=True)

In [ ]:
state_ids = {k: i for i, k in enumerate(gdf_pb.name_1)}
state_ids

In [ ]:
shapes = zip(gdf_pb.geometry.buffer(0), range(len(gdf_pb)))

In [ ]:
ds = xds.copy()
ds['states'] = rasterize(shapes, ds.coords, longitude='lon', latitude='lat')

In [ ]:
ds

In [ ]:
ds.states.plot.imshow()

In [ ]:
(ds.states == state_ids['Corrientes']).plot.imshow()

In [ ]:
ds

In [ ]:
(ds.stocks
 .isel(time=slice(12))
 .where(ds.states == state_ids['Corrientes'])
 .sel(lon=slice(-62, -55), lat=slice(-32, -25))
 .plot.imshow(col='time', col_wrap=4))